# Using the Neptune Gremlin Client in an AWS Lambda function

![Architecture: Using the Neptune Gremlin Client in an AWS Lambda function](https://ianrob-examples.s3-eu-west-1.amazonaws.com/images/gremlin-client-lambda.png "Architecture: Using the Neptune Gremlin Client in an AWS Lambda function")

In [ ]:
import boto3
import os
import json

client = boto3.client('lambda')

## neptune-endpoint-info Lambda function

This Lambda function acts as a proxy for a `ClusterEndpointsRefreshAgent`. The Lambda polls the Neptune management API on a periodic basis to fetch all the available instance endpoints. Lambda functions that use a `GremlinClient` with a `ClusterEndpointsRefreshAgent` can then use this Lambda to return specific endpoints (`All`, `Primary` or `ReadReplicas`).

Use of this Lambda ensures that the management API is not throttled, irrespective of the number of concurrent application Lambdas.

The source code for this function can be found [here](https://github.com/awslabs/amazon-neptune-tools/blob/master/neptune-gremlin-client/neptune-endpoints-info-lambda/src/main/java/software/amazon/lambda/NeptuneEndpointsInfoLambda.java). There's a CloudFormation template that you can use to install the Lamdba [here](https://github.com/awslabs/amazon-neptune-tools/blob/master/neptune-gremlin-client/cloudformation-templates/neptune-endpoints-info-lambda.json).

In [ ]:
endpoint_info = client.invoke(
    FunctionName='neptune-endpoint-info_{}'.format(os.environ['NEPTUNE_CLUSTER_ID']),
    InvocationType='RequestResponse',
    Payload=b'"All"' # or 'Primary' or 'ReadReplicas'
)

print(endpoint_info['Payload'].read().decode("utf-8"))

## neptune-gremlin-client-example Lambda function

This Lambda function uses a `GremlinClient` to query Neptune. With each invocation it creates a new vertex. The function uses a `ClusterEndpointsRefreshAgent` that is configured to retrieve the Primary endpoint from the _neptune-endpoint-info_ described above.

The source code for this demo function can be found [here](https://github.com/awslabs/amazon-neptune-tools/blob/master/neptune-gremlin-client/gremlin-client-demo/src/main/java/software/amazon/lambda/NeptuneGremlinClientExampleLambda.java).

In [ ]:
gremlin_client_lambda_response = client.invoke(
    FunctionName='neptune-gremlin-client-example_{}'.format(os.environ['NEPTUNE_CLUSTER_ID']),
    InvocationType='RequestResponse'
)

print('New vertex: {}'.format(gremlin_client_lambda_response['Payload'].read().decode("utf-8")))